In [4]:
import os,sys
sys.path.insert(1, '../')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import seaborn as sns
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from scipy import sparse
from tqdm import tqdm
import pdb

from poisson_utils import get_data, get_tree, acs_filter, transform_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Year and whether to transform the data
curr_year = 2019
transform = False
# Feature codes to use
feature_names =['PINCP','AGEP', 'PUBCOV','PRIVCOV','MAR','SCH','DIS','ESP','CIT','MIG','MIL','ANC','NATIVITY','DEAR','DEYE','DREM','SEX','RAC1P']
ft = np.array(["q", "q", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c"])
regression_feature_name = "PINCP"
outcome_name = "SCHL"
features, outcome = get_data(curr_year, feature_names, outcome_name, regression_feature_name, acs_filter=acs_filter)
reg_feat = features[regression_feature_name]
if transform:
    features, enc = transform_features(features, ft)
else:
    enc = None

np.random.seed(0) # Fix seed for tree
tree = get_tree(curr_year-1, feature_names, ft, outcome_name, regression_feature_name, enc=enc, transform=transform, acs_filter=acs_filter)
np.random.seed(0) # Fix seed for evaluation
# Evaluate tree and plot data
predicted_outcome = tree.predict(xgb.DMatrix(features))

Training tree without transformation.
[0]	eval-error:-16.72059	eval-mae:3796.39115	train-error:-16.70589	train-mae:3796.40582
[1]	eval-error:-16.72059	eval-mae:3612.45043	train-error:-16.70589	train-mae:3612.46500
[2]	eval-error:-16.72059	eval-mae:3437.42101	train-error:-16.70589	train-mae:3437.43552
[3]	eval-error:-16.72059	eval-mae:3270.87049	train-error:-16.70589	train-mae:3270.88501
[4]	eval-error:-16.72059	eval-mae:3112.38886	train-error:-16.70589	train-mae:3112.40331
[5]	eval-error:-16.72059	eval-mae:2961.58532	train-error:-16.70589	train-mae:2961.59961
[6]	eval-error:-16.72059	eval-mae:2818.08799	train-error:-16.70589	train-mae:2818.10193
[7]	eval-error:-16.72059	eval-mae:2681.54224	train-error:-16.70589	train-mae:2681.55611
[8]	eval-error:-16.72059	eval-mae:2551.61161	train-error:-16.70589	train-mae:2551.62521
[9]	eval-error:-16.72059	eval-mae:2427.97537	train-error:-16.70589	train-mae:2427.98895


/Users/angelopoulos/miniconda3/lib/python3.12/site-packages/xgboost/core.py:723: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[10]	eval-error:-16.72059	eval-mae:2310.32913	train-error:-16.70589	train-mae:2310.34240
[11]	eval-error:-16.72059	eval-mae:2198.38231	train-error:-16.70589	train-mae:2198.39529
[12]	eval-error:-16.72059	eval-mae:2091.85885	train-error:-16.70589	train-mae:2091.87145
[13]	eval-error:-16.72059	eval-mae:1990.49583	train-error:-16.70589	train-mae:1990.50842
[14]	eval-error:-16.72059	eval-mae:1894.04325	train-error:-16.70589	train-mae:1894.05588
[15]	eval-error:-16.72059	eval-mae:1802.26382	train-error:-16.70589	train-mae:1802.27623
[16]	eval-error:-16.72059	eval-mae:1714.93055	train-error:-16.70589	train-mae:1714.94285
[17]	eval-error:-16.72059	eval-mae:1631.82833	train-error:-16.70589	train-mae:1631.84046
[18]	eval-error:-16.72059	eval-mae:1552.75224	train-error:-16.70589	train-mae:1552.76408
[19]	eval-error:-16.72059	eval-mae:1477.50686	train-error:-16.70589	train-mae:1477.51842
[20]	eval-error:-16.72059	eval-mae:1405.90691	train-error:-16.70589	train-mae:1405.91813
[21]	eval-error:-16.7

In [29]:
print(outcome.var(), (outcome-predicted_outcome).var())

19.234669336599516 11.697047437212849


In [31]:
X = features[['PINCP', 'SEX']].to_numpy()
X = np.hstack((X, np.ones((X.shape[0], 1))))
X[:,1] = X[:,1] * (np.linalg.norm(X, axis=0)[0] / np.linalg.norm(X, axis=0)[1])
X[:,2] = X[:,2] * (np.linalg.norm(X, axis=0)[0] / np.linalg.norm(X, axis=0)[2])
Y = outcome.to_numpy()
Yhat = predicted_outcome

In [33]:
os.makedirs('./data', exist_ok=True)
np.savez(
    './data/census_education.npz',
    X=X,
    Yhat=Yhat,
    Y=Y,
)